### Getting data from yann lecunn dataset

Yann Lecunn dataset is of the format idx and split across 4 files
1. train-images.idx3-ubyte : training image set which consists of 60000 images each image is represented by a 28*28 array
2. train-labels.idx1-ubyte : training label set which consists of 60000 labels 
3. t10k-images.idx3-ubyte : test image set which consists of 10000 images each image is represented by a 28*28
4. t10k-labels.idx1-ubyte : training label set which consists of 10000 labels

In [1]:
import idx2numpy
import seaborn as sns

#### Extract Training data

In [2]:
X_train_3D = idx2numpy.convert_from_file('train-images.idx3-ubyte')
X_train = X_train_3D.flatten().reshape(60000,784)

y_train = idx2numpy.convert_from_file('train-labels.idx1-ubyte')
X_train.shape, y_train.shape

((60000, 784), (60000,))

#### Extract Test data

In [3]:
X_test_3D = idx2numpy.convert_from_file('t10k-images.idx3-ubyte')
X_test =  X_test_3D.flatten().reshape(10000,784)

y_test = idx2numpy.convert_from_file('t10k-labels.idx1-ubyte')
X_test.shape, y_test.shape

((10000, 784), (10000,))

# Decision Tree

In [4]:
import numpy as np
from sklearn.utils import shuffle
X_shuffle,y_shuffle = shuffle(X_train,y_train)
X_train = X_shuffle[0:50000]
y_train = y_shuffle[0:50000]

from sklearn import tree
from sklearn.model_selection import cross_val_predict

#dt_clf = tree.DecisionTreeClassifier(max_depth=20, max_leaf_nodes=300)
#dt_clf = tree.DecisionTreeClassifier(max_depth=4, max_leaf_nodes=20)
dt_clf = tree.DecisionTreeClassifier(max_depth=8)

#y_train_pred = cross_val_predict(dt_clf, X_train, y_train, cv=2)
print(dt_clf.fit(X_train, y_train))

print('Number of nodes in the decision tree {}.'.format(dt_clf.tree_.node_count))
print('Number of threshold in the decision tree {}.'.format(len(dt_clf.tree_.threshold)))
print('Number of leaves in the decision tree {}.'.format(dt_clf.tree_.n_leaves))

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, dt_clf.predict(X_test)), accuracy_score(y_train, dt_clf.predict(X_train)))

DecisionTreeClassifier(max_depth=8)
Number of nodes in the decision tree 505.
Number of threshold in the decision tree 505.
Number of leaves in the decision tree 253.
0.8141 0.82696


In [5]:
threshold = dt_clf.tree_.threshold
import pandas as pd
df_train = pd.DataFrame(data = X_train, columns = range(X_train[0].shape[0]))
df_test = pd.DataFrame(data = X_test, columns = range(X_test[0].shape[0]))
df_train.shape, df_test.shape
df = pd.concat([df_train, df_test])
unique_vals = []
for i in df.columns:
    unique_vals.append(df[i].unique())
flatten_list = np.concatenate(unique_vals).ravel()
print('Max and Min values of thresholds in decision tree are', max([int(i) for i in list(set(threshold))]), min([int(i) for i in list(set(threshold))]))

Max and Min values of thresholds in decision tree are 253 -2


### Displaying the tree

In [6]:
from sklearn.tree import export_text
text_representation = export_text(dt_clf)
print(text_representation, dt_clf.classes_)

|--- feature_350 <= 131.50
|   |--- feature_568 <= 0.50
|   |   |--- feature_430 <= 0.50
|   |   |   |--- feature_405 <= 16.50
|   |   |   |   |--- feature_484 <= 0.50
|   |   |   |   |   |--- feature_154 <= 0.50
|   |   |   |   |   |   |--- feature_594 <= 21.50
|   |   |   |   |   |   |   |--- feature_157 <= 0.50
|   |   |   |   |   |   |   |   |--- class: 7
|   |   |   |   |   |   |   |--- feature_157 >  0.50
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- feature_594 >  21.50
|   |   |   |   |   |   |   |--- feature_302 <= 6.00
|   |   |   |   |   |   |   |   |--- class: 5
|   |   |   |   |   |   |   |--- feature_302 >  6.00
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- feature_154 >  0.50
|   |   |   |   |   |   |--- feature_509 <= 22.50
|   |   |   |   |   |   |   |--- feature_544 <= 4.50
|   |   |   |   |   |   |   |   |--- class: 3
|   |   |   |   |   |   |   |--- feature_544 >  4.50
|   |   |   |   |   |   |   |   |--- class: 

### Sequential: New FSM Generator

In [8]:
%%capture cap --no-stderr
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    
    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print ("{}{}:if ({} <= {})".format(indent, node, name, int(round(threshold,3)))) 
            recurse(tree_.children_left[node], depth + 1)
            print ("{}{}:else ".format(indent, node, name, int(round(threshold,3))))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print ("{} Label<={};".format(indent, np.argmax(tree_.value[node][0],axis=0)))

    recurse(0, 1)

cols = range(784)
features = ['pixels[{}]'.format(str(i)) for i in cols]
class_names = [str(i) for i in dt_clf.classes_]
tree_to_code(dt_clf, features)

with open('verilog_newFSM.txt', 'w') as f:
    f.write(cap.stdout)

In [10]:
import pdb
def write_file(data_towrite):
    with open('verilog_file.v', 'a', encoding='utf-8') as file:
        file.writelines(data_towrite)
        file.close()
        
def write_line_file(data_towrite, line_num):
    with open('verilog_file.v', 'r', encoding='utf-8') as file:
        file_content = file.readlines()
        file.close()
    file_content[line_num] = file_content[line_num].replace('\n','') + ' ' + data_towrite + '\n'  
    with open('verilog_file.v', 'w', encoding='utf-8') as file:
        file.writelines(file_content)
        file.close()
        
def search_content_file(word):
    with open('verilog_file.v', 'r') as file:
        file_content = file.readlines()
        for line_num,line in enumerate(file_content):
            if word in line:
                return(line_num)
        return("content doesn't exists.")
    
def update_stateformat_file():
    with open('verilog_file.v', 'r', encoding='utf-8') as file:
        file_content = file.readlines()
        file.close()
    for line_num,line in enumerate(file_content):
        if 'if' in line:
            file_content[line_num] = '16\'d' + file_content[line_num].strip() + ' \n'  
    with open('verilog_file.v', 'w', encoding='utf-8') as file:
        file.writelines(file_content)
        file.close()
        
def clear_file():
    with open('verilog_file.v', 'w', encoding='utf-8') as file:
        pass
        file.close()

import itertools
with open('verilog_newFSM.txt', 'r') as f:
    tree_verilog = f.read()
    f.close()

clear_file()
line_num = 0
else_state = 0
else_nextif_state = 0
curr_line,next_line = itertools.tee(tree_verilog.split('\n'))
next(next_line, None)
for i,j in list(zip(curr_line,next_line)):
    if 'Label' in i:
        pass
        #write_file(f'  begin {i.strip()} state<=0;ml_inference_completed<=1; end \n')
    elif 'else' in i and 'Label' in j:
        else_state = i.strip().split(':')[0]
        line_num = search_content_file(' '+str(else_state)+':if' )
        write_line_file(f'\n       else begin {j.strip()} state<=0;ml_inference_completed<=1; end ', line_num)
        #write_file(f'   {i.strip().split(":")[-1]} ')
    elif 'else' in i and 'if' in j:
        else_state = i.strip().split(':')[0]
        else_nextif_state = j.strip().split(':')[0]
        line_num = search_content_file(' '+str(else_state)+':if' )
        write_line_file(f'else begin state<={else_nextif_state}; end ', line_num)
    elif 'if' in i and 'Label' in j:
        write_file(f'\n {i.strip()} begin {j.strip()} state<=0;ml_inference_completed<=1; end ')
    elif 'if' in i and 'if' in j:
        write_file(f'\n {i.strip()} begin state<={j.strip().split(":")[0]}; end \n ')  
update_stateformat_file()

### Verifying the number of "if", "else" and "Label" in verilog with decision tree architecture

In [11]:
#verifying that the number of "if" statements is one less than the number of nodes.
file1 = open('verilog_file.v', 'r')
contents = file1.readlines()
counter_if =0
for line in contents:
    if 'if' in line:
        counter_if = counter_if+1
print('Number of "if" statements {}.'.format(counter_if))

#verifying that the number of "else" statements is one less than the number of leaves.
file1 = open('verilog_file.v', 'r')
contents = file1.readlines()
counter_else =0
for line in contents:
    if 'else' in line:
        counter_else = counter_else+1
print('Number of "else" statements {}.'.format(counter_else))

#verifying that the number of "Label" statements is equal to the number of leaves.
file1 = open('verilog_file.v', 'r')
contents = file1.readlines()
counter_return =0
for line in contents:
    if 'Label' in line:
        counter_return = counter_return+1
print('Number of "label" statements {}.'.format(counter_return))

Number of "if" statements 252.
Number of "else" statements 252.
Number of "label" statements 253.


### Convert the tree into the source code by rounding the threshold to nearest integer and save it in a txt file. This is performed to verify the accuracy by using this dumped decision rules.

In [13]:
%%capture cap --no-stderr
#https://towardsdatascience.com/scikit-learn-decision-trees-explained-803f3812290d
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print ("def decision_tree_inference({}):".format('feature_set'))
    for i,pixel in enumerate(feature_names):
            print ("{}{}".format("  ", pixel+'='+'feature_set['+str(i)+']'))
    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print ("{}if {} <= {}:".format(indent, name, int(round(threshold,3))))  #convert the threshold to integer
            recurse(tree_.children_left[node], depth + 1)
            print ("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print ("{}return {}".format(indent, np.argmax(tree_.value[node][0],axis=0)))

    recurse(0, 1)

cols = range(784)
features = ['pixel'+str(i) for i in cols]
class_names = [str(i) for i in dt_clf.classes_]
tree_to_code(dt_clf, features)

with open('mnist_decision_tree_inference.py', 'w') as f:
    f.write(cap.stdout)

In [14]:
from mnist_decision_tree_inference import decision_tree_inference
y_test_pred_tree = []
for i,test_samples in enumerate(X_test):
    y_test_pred_tree.append(decision_tree_inference(test_samples))

y_train_pred_tree = []
for i,test_samples in enumerate(X_train):
    y_train_pred_tree.append(decision_tree_inference(test_samples))

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_test_pred_tree), accuracy_score(y_train, y_train_pred_tree))

0.8141 0.82696


### OPE Encryption

In [15]:
#OPE_ciphers = {0: 12066, 1: 169498, 2: 226543, 3: 672042, 4: 838658, 5: 919684, 6: 977270, 7: 1055701, 8: 1058312, 9: 1087796, 10: 1097445, 11: 1176451, 12: 1183921, 13: 1220952, 14: 1557525, 15: 1727241, 16: 1892922, 17: 1960170, 18: 1966351, 19: 2025917, 20: 2086019, 21: 2186547, 22: 2240207, 23: 2247658, 24: 2262361, 25: 2322390, 26: 2400481, 27: 2456156, 28: 2492957, 29: 2566854, 30: 2641406, 31: 2654827, 32: 2657422, 33: 2673702, 34: 2710579, 35: 2896614, 36: 2970776, 37: 3006623, 38: 3015638, 39: 3041948, 40: 3127827, 41: 3150608, 42: 3187553, 43: 3216337, 44: 3344121, 45: 3442649, 46: 3511024, 47: 3594918, 48: 3655763, 49: 3672413, 50: 4035862, 51: 4081037, 52: 4104042, 53: 4152186, 54: 4163483, 55: 4180647, 56: 4246894, 57: 4351195, 58: 4429184, 59: 4549356, 60: 4607063, 61: 4639927, 62: 4782698, 63: 4788482, 64: 4837431, 65: 4864153, 66: 5118423, 67: 5261392, 68: 5299828, 69: 5367820, 70: 5389522, 71: 5404390, 72: 5440993, 73: 5521457, 74: 5571806, 75: 5576710, 76: 5603955, 77: 5623693, 78: 5687500, 79: 5784075, 80: 5801917, 81: 5815677, 82: 5860302, 83: 5895894, 84: 5910480, 85: 5962880, 86: 5987354, 87: 6025885, 88: 6031446, 89: 6045911, 90: 6214893, 91: 6269128, 92: 6320478, 93: 6325746, 94: 6358493, 95: 6382120, 96: 6414249, 97: 6469510, 98: 6517938, 99: 6553015, 100: 6586802, 101: 6657083, 102: 6766826, 103: 6824707, 104: 6851836, 105: 6887293, 106: 6901880, 107: 6922412, 108: 6945741, 109: 7088186, 110: 7169022, 111: 7387426, 112: 7498208, 113: 7574683, 114: 7877256, 115: 7990376, 116: 8014774, 117: 8247426, 118: 8310828, 119: 8348065, 120: 8420586, 121: 8447951, 122: 8484928, 123: 8573440, 124: 8592862, 125: 8606928, 126: 8675181, 127: 8693099, 128: 8800638, 129: 8817217, 130: 8874431, 131: 8883869, 132: 8920338, 133: 8942168, 134: 8959790, 135: 8966065, 136: 9099182, 137: 9115343, 138: 9201113, 139: 9268521, 140: 9307567, 141: 9345791, 142: 9375555, 143: 9377486, 144: 9394318, 145: 9398866, 146: 9458713, 147: 9541748, 148: 9619426, 149: 9676186, 150: 9715339, 151: 9757607, 152: 9804769, 153: 9857918, 154: 9891099, 155: 9903648, 156: 10010619, 157: 10038400, 158: 10138825, 159: 10168981, 160: 10275301, 161: 10385488, 162: 10486337, 163: 10487008, 164: 10620586, 165: 10655005, 166: 10680653, 167: 10694574, 168: 10724530, 169: 10779532, 170: 10828196, 171: 10830169, 172: 10848005, 173: 10876729, 174: 10926801, 175: 10954040, 176: 10976026, 177: 10977614, 178: 11056348, 179: 11250015, 180: 11302657, 181: 11315752, 182: 11460622, 183: 11536965, 184: 11572194, 185: 11677613, 186: 11698564, 187: 11743495, 188: 11783171, 189: 11911512, 190: 11937653, 191: 11947496, 192: 11976429, 193: 12012962, 194: 12031102, 195: 12109032, 196: 12238082, 197: 12280297, 198: 12295705, 199: 12377556, 200: 12388676, 201: 12476849, 202: 12502820, 203: 12576668, 204: 12630390, 205: 12698018, 206: 12724395, 207: 12777658, 208: 12876195, 209: 12960605, 210: 12993132, 211: 13034518, 212: 13225130, 213: 13251178, 214: 13278072, 215: 13425223, 216: 13577682, 217: 13650047, 218: 13760126, 219: 13867767, 220: 13949008, 221: 14074481, 222: 14138661, 223: 14156903, 224: 14327531, 225: 14364236, 226: 14391815, 227: 14533063, 228: 14584445, 229: 14692083, 230: 14727026, 231: 14785119, 232: 14850983, 233: 14965647, 234: 15038435, 235: 15059755, 236: 15207521, 237: 15280679, 238: 15305829, 239: 15337248, 240: 15415187, 241: 15423422, 242: 15551713, 243: 15791922, 244: 15802191, 245: 15861718, 246: 16000222, 247: 16069522, 248: 16267485, 249: 16306400, 250: 16537043, 251: 16637700, 252: 16730424, 253: 17031859, 254: 17167896, 255: 17228611}
import pdb
unique_thresholds = list(set([int(round(i,3)) for i in threshold]))
print(f'Min value of threshold = {min(unique_thresholds)}. Max value of threshold = {max(unique_thresholds)}')
print('Unique values in thresholds are: ', unique_thresholds)
from pyope.ope import OPE, ValueRange
#random_key = OPE.generate_key()
cipher = OPE(b'long key' * 2, in_range=ValueRange(-100, 1000),
                              out_range=ValueRange(0, 100000000))
OPE_ciphers = {}
all_plaintexts = list( set(unique_thresholds).union(set(range(256))) ) #Taking all the thresholds and pixels values
all_plaintexts.sort()
for i in all_plaintexts:
    OPE_ciphers[i] = cipher.encrypt(i)

    
flag = 0
ciphers_indecimal_copy = list(OPE_ciphers.values())[:]
ciphers_indecimal_copy.sort()
if (ciphers_indecimal_copy == list(OPE_ciphers.values())):
    flag = 1
     
# printing result
if (flag) :
    print ("Yes, ciphers are sorted. OPE valid.")
else :
    print ("No, ciphers are not sorted. OPE invalid.")

Min value of threshold = -2. Max value of threshold = 253
Unique values in thresholds are:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 26, 27, 29, 30, 31, 34, 35, 36, 41, 42, 43, 44, 46, 47, 48, 51, 52, 53, 57, 58, 59, 60, 62, 65, 68, 70, 71, 77, 78, 79, 82, 84, 87, 88, 90, 92, 93, 94, 96, 97, 101, 104, 108, 109, 110, 113, 114, 115, 119, 121, 124, 126, 131, 133, 134, 135, 138, 150, 153, 156, 159, 162, 164, 166, 168, 179, 183, 192, 198, 203, 205, 209, 213, 214, 222, 224, 225, 226, 231, 234, 246, 248, 251, 252, 253, -2]
Yes, ciphers are sorted. OPE valid.


In [36]:
print(f'Key Display\n In string: {cipher.generate_key().decode()} \n In Hex: {cipher.generate_key().hex()} \n In integer: {int(cipher.generate_key().hex(), base=16)}')
print(OPE_ciphers)

Key Display
 In string: PPu3WGQj3+gyfEsARIJWA19oseVph2qjQ0Ok19K3b6Q= 
 In Hex: 30547864764539542b743861714361316c754e43316754743575544a4c6f54324442723467462b6446736f3d 
 In integer: 4020262153119643782837989383967985752122808315360222786683360755567309839741158713029944769305612277411645
{-2: 9913043, 0: 10051064, 1: 10110880, 2: 10178451, 3: 10239092, 4: 10256163, 5: 10311743, 6: 10428057, 7: 10459335, 8: 10495495, 9: 10621710, 10: 10645813, 11: 10700220, 12: 10708006, 13: 10738547, 14: 10810050, 15: 10847584, 16: 10957410, 17: 10964188, 18: 11365368, 19: 11588596, 20: 11667110, 21: 11763668, 22: 11885383, 23: 12053585, 24: 12113758, 25: 12161483, 26: 12207861, 27: 12220718, 28: 12228678, 29: 12411484, 30: 12880615, 31: 12908743, 32: 12953385, 33: 13090980, 34: 13115331, 35: 13178136, 36: 13229076, 37: 13242460, 38: 13330733, 39: 13419802, 40: 13445220, 41: 13490312, 42: 13636489, 43: 13677650, 44: 13759948, 45: 13869191, 46: 13873290, 47: 13978160, 48: 13999815, 49: 14003727, 50: 142

### Encrypted: Sequential: New FSM Generator

In [25]:
%%capture cap --no-stderr
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    
    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print ("{}{}:if ({} <= {})".format(indent, node, name, OPE_ciphers[int(round(threshold,3))]))
            recurse(tree_.children_left[node], depth + 1)
            print ("{}{}:else ".format(indent, node))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print ("{} Label<={};".format(indent, OPE_ciphers[np.argmax(tree_.value[node][0],axis=0)]))

    recurse(0, 1)

cols = range(784)
features = ['Encrypted_pixels['+str(i)+']' for i in cols]
class_names = [str(OPE_ciphers[i]) for i in dt_clf.classes_]
tree_to_code(dt_clf, features)

with open('verilog_newFSM_enc.txt', 'w') as f:
    f.write(cap.stdout)

In [26]:
def write_file(data_towrite):
    with open('verilog_file_enc.v', 'a', encoding='utf-8') as file:
        file.writelines(data_towrite)
        file.close()
        
def write_line_file(data_towrite, line_num):
    with open('verilog_file_enc.v', 'r', encoding='utf-8') as file:
        file_content = file.readlines()
        file.close()
    file_content[line_num] = file_content[line_num].replace('\n','') + ' ' + data_towrite + '\n'  
    with open('verilog_file_enc.v', 'w', encoding='utf-8') as file:
        file.writelines(file_content)
        file.close()
        
def update_stateformat_file():
    with open('verilog_file_enc.v', 'r', encoding='utf-8') as file:
        file_content = file.readlines()
        file.close()
    for line_num,line in enumerate(file_content):
        if 'if' in line:
            file_content[line_num] = '16\'d' + file_content[line_num].strip() + ' \n'  
    with open('verilog_file_enc.v', 'w', encoding='utf-8') as file:
        file.writelines(file_content)
        file.close()
        
def search_content_file(word):
    with open('verilog_file_enc.v', 'r') as file:
        file_content = file.readlines()
        for line_num,line in enumerate(file_content):
            if word in line:
                return(line_num)
        return("content doesn't exists.")
        
def clear_file():
    with open('verilog_file_enc.v', 'w', encoding='utf-8') as file:
        pass
        file.close()

import itertools
with open('verilog_newFSM_enc.txt', 'r') as f:
    tree_verilog = f.read()
    f.close()

clear_file()
line_num = 0
else_state = 0
else_nextif_state = 0
curr_line,next_line = itertools.tee(tree_verilog.split('\n'))
next(next_line, None)
for i,j in list(zip(curr_line,next_line)):
    if 'Label' in i:
        pass
        #write_file(f'  begin {i.strip()} state<=0;ml_inference_completed<=1; end \n')
    elif 'else' in i and 'Label' in j:
        else_state = i.strip().split(':')[0]
        line_num = search_content_file(' '+str(else_state)+':if' )
        write_line_file(f'\n       else begin {j.strip()} state<=0;ml_inference_completed<=1; end ', line_num)
        #write_file(f'   {i.strip().split(":")[-1]} ')
    elif 'else' in i and 'if' in j:
        else_state = i.strip().split(':')[0]
        else_nextif_state = j.strip().split(':')[0]
        line_num = search_content_file(' '+str(else_state)+':if' )
        write_line_file(f'else begin state<={else_nextif_state}; end ', line_num)
    elif 'if' in i and 'Label' in j:
        write_file(f'\n {i.strip()} begin {j.strip()} state<=0;ml_inference_completed<=1; end ')
    elif 'if' in i and 'if' in j:
        write_file(f'\n {i.strip()} begin state<={j.strip().split(":")[0]}; end \n ')  
update_stateformat_file()

### Verifying the number of "if", "else" and "Label" in verilog with decision tree architecture

In [27]:
#verifying that the number of "if" statements is one less than the number of nodes.
file1 = open('verilog_file_enc.v', 'r')
contents = file1.readlines()
counter_if =0
for line in contents:
    if 'if' in line:
        counter_if = counter_if+1
print('Number of "if" statements {}.'.format(counter_if))

#verifying that the number of "else" statements is one less than the number of leaves.
file1 = open('verilog_file_enc.v', 'r')
contents = file1.readlines()
counter_else =0
for line in contents:
    if 'else' in line:
        counter_else = counter_else+1
print('Number of "else" statements {}.'.format(counter_else))

#verifying that the number of "Label" statements is equal to the number of leaves.
file1 = open('verilog_file_enc.v', 'r')
contents = file1.readlines()
counter_return =0
for line in contents:
    if 'Label' in line:
        counter_return = counter_return+1
print('Number of "label" statements {}.'.format(counter_return))

Number of "if" statements 252.
Number of "else" statements 252.
Number of "label" statements 253.
